In [1]:
import pandas as pd
import numpy as np
from src.utils.defect_utils import featureset
import src.utils.import_pickle as ip
### 20190925 - calibrated - only defects
### 20190128 - insulation
"""
old_filenames=['gDFZ_2019-05-27T08_55_55_819Z_DFZ01', # s
               'gDFZ_2019-05-27T10_03_59_077Z_DFZ01', # t - worked with this
               'gDFZ_2019-05-27T13_05_53_330Z_DFZ01', # u
               'gDFZ_2019-05-27T14_10_51_425Z_DFZ01'] # v"""

new_filenames = ['__new_gDFZ_2019-05-27T08_55_55_819Z_DFZ01', # s - 819Z_DFZ01
                 '__new_gDFZ_2019-05-27T10_03_59_077Z_DFZ01', # t - 077Z_DFZ01
                 '__new_gDFZ_2019-05-27T13_05_53_330Z_DFZ01', # u - 330Z_DFZ01
                 '__new_gDFZ_2019-05-27T14_10_51_425Z_DFZ01'] # v - 425Z_DFZ01


s = ip.import_pickle(new_filenames[0]) # not that good
t = ip.import_pickle(new_filenames[1])
u = ip.import_pickle(new_filenames[2])
v = ip.import_pickle(new_filenames[3])

print("#################### S ####################")
s_features = featureset(s)
print("#################### T ####################")
t_features = featureset(t)
print("#################### U ####################")
u_features = featureset(u)
print("#################### V ####################")
v_features = featureset(v)

Importing Pickle: __new_gDFZ_2019-05-27T08_55_55_819Z_DFZ01.pickle
Warning, the name:  HOL 303.2 - 9005_D5.1  is tranformed to: 
HOL 303.2-9005D5.1
Warning, the name:  BN 42_K6.2 - 43.2  is tranformed to: 
BN 42K6.2-43.2
Warning, the name:  BN 42_K6.1-2  is tranformed to: 
BN 42K6.1-2
Warning, the name:  BN 38_K6.3 - 42.3  is tranformed to: 
BN 38K6.3-42.3
Warning, the name:  BN 38_K6.3-4  is tranformed to: 
BN 38K6.3-4
Warning, the name:  BN 41_K5.1 - 41.1  is tranformed to: 
BN 41K5.1-41.1
Warning, the name:  BN 41_K5.1-2  is tranformed to: 
BN 41K5.1-2
Warning, the name:  BN 15.3 - 41_K5.2  is tranformed to: 
BN 15.3-41K5.2
Importing Pickle: __new_gDFZ_2019-05-27T10_03_59_077Z_DFZ01.pickle
Importing Pickle: __new_gDFZ_2019-05-27T13_05_53_330Z_DFZ01.pickle
Importing Pickle: __new_gDFZ_2019-05-27T14_10_51_425Z_DFZ01.pickle


ZMON AXLE_11:   0%|          | 0/87 [00:00<?, ?it/s]

Warning, the name:  BN 2_K6.1-2  is tranformed to: 
BN 2K6.1-2
#################### S ####################


ZMON AXLE_11:   0%|          | 0/56 [00:00<?, ?it/s]

#################### T ####################


ZMON AXLE_11:   0%|          | 0/74 [00:00<?, ?it/s]

#################### U ####################


ZMON AXLE_11:   0%|          | 0/99 [00:00<?, ?it/s]

#################### V ####################


insulationjoint AXLE_42: 100%|██████████| 130/130 [00:22<00:00,  5.78it/s]


In [2]:
from src.utils.defect_utils import save_pickle
save_pickle([s_features, t_features, u_features, v_features], 'new')